In [1]:
import torch 
import os 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

In [15]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 64
opt.gpu_num = 0
opt.model = "csn"
opt.model_depth = 50
opt.version = '0719' + '_' + opt.model + '_depth' + str(opt.model_depth)


In [16]:
from utils.utils import get_ckpt_path

model = WalkVideoClassificationLightningModule(opt)

# get last ckpt path
ckpt_path = get_ckpt_path(opt)

model = model.load_from_checkpoint(ckpt_path)

model.eval()

clear_output()


In [18]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)


trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=-1,
)

trainer.test(dataloaders=module, model=model)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_acc_step       │    0.35428571701049805    │
│      test_dice_step       │    0.35428571701049805    │
│         test_loss         │     6.669982433319092     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 6.669982433319092,
  'test_acc_step': 0.35428571701049805,
  'test_dice_step': 0.35428571701049805}]

In [ ]:
input_data = next(iter(test_data))


In [ ]:
input_data['video'].shape

input_data["video_name"]

In [ ]:
input_data['label']

In [ ]:
preds = model(input_data['video'])

In [ ]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [ ]:
pred_classes

In [ ]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [ ]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [ ]:
real_calss

In [ ]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

In [ ]:
pred_class_names == real_calss

In [ ]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result